### Manipulation of Comet database search results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Comet search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Search database included marine fungi and labrinthulomyces discovered using de novo peptide sequencing
    XInteract file includes precursor intensities and protein descriptions

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported Comet search results after running through XInteract and saving as `.xlsx` files (pandas really doesn't like to read the `xls` verions of the XInteract output becase there are so many characters in the `protein` column for these samples) into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls

231/  233/  243/  273/  278/  378/


In [4]:
cd 278/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278


In [5]:
ls

ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_NAAF.csv
ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_NAAF_totals.csv
ETNP-SKQ17-278-265m-trap_PTM-opt_Comet15_1FDR_stripped_peptides.fas
ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_stripped_peptides.txt
ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_totals.csv
ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_fungidb.csv
trap_265m_15ppm_quant.xlsx


## 278: 265m trap filtered on 0.3 um GF-75

In [6]:
# read the CSV into a dataframe using the pandas read_excel function

cometdup278 = pd.read_excel("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/trap_265m_15ppm_quant.xlsx")

# remove redundant rows
comet278 = pd.DataFrame.drop_duplicates(cometdup278)

print("# redundant Comet peptides in combined dataframe", len(cometdup278))
print("# nonredundant Comet peptides in combined dataframe", len(comet278))

#look at the dataframe
comet278.head()

# redundant Comet peptides in combined dataframe 32832
# nonredundant Comet peptides in combined dataframe 32832


,spectrum,xcorr,deltacn,expect,ions,peptide,calc_neutral_pep_mass,precursor_intensity,protein
0,20181004_4-19_265m_topP_278_90min.45087.45087.2,5.642,0.615,7.440000e-14,27/44,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,2215090.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
1,20181004_4-19_265m_topP_278_90min.45016.45016.3,5.634,0.503,1.350000e-08,29/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,13100100.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
2,20181004_4-19_265m_topP_278_90min.24694.24694.2,4.085,0.420,3.220000e-05,22/36,R.LPQVEGTGGDVQPSQDLVR.V,1994.0069,18697900.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
3,20181004_4-19_265m_topP_278_90min.25494.25494.2,3.972,0.463,1.230000e-06,22/36,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,1994.9909,2532930.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
4,20181004_4-19_265m_topP_278_90min.45263.45263.3,3.884,1.000,8.030000e-12,19/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,376172.0,gi|54036848|sp|P63284.1|CLPB_ECOLI


In [7]:
# get rid of rows where the xcorr is unavailable (usually 3 or so)
comet278 = comet278[comet278.xcorr != '[unavailable]']

# use str.strip with indexing by str[0] to add a column with the peptide's left terminus
comet278['L terminus'] = comet278['peptide'].astype(str).str[0]

# use str.strip with indexing by str[-1] to add a column with the peptide's left terminus
comet278['R terminus'] = comet278['peptide'].str.strip().str[-1]

# use a count function to enumerate the # of A's (alanines) in each peptide
comet278['A'] = comet278['peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
comet278['C'] = comet278['peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
comet278['D'] = comet278['peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
comet278['E'] = comet278['peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
comet278['F'] = comet278['peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
comet278['G'] = comet278['peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
comet278['H'] = comet278['peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in comet278 output, there will be no isoleucines (they're lumped in with leucines)
comet278['I'] = comet278['peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
comet278['K'] = comet278['peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
comet278['L'] = comet278['peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
comet278['M'] = comet278['peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
comet278['N'] = comet278['peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
comet278['P'] = comet278['peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
comet278['Q'] = comet278['peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
comet278['R'] = comet278['peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
comet278['S'] = comet278['peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
comet278['T'] = comet278['peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
comet278['V'] = comet278['peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
comet278['W'] = comet278['peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
comet278['Y'] = comet278['peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
comet278['c-carb'] = comet278['peptide'].str.count("160.03")

# use a count function to enumerate the # of oxidized M's in each peptide
comet278['m-oxid'] = comet278['peptide'].str.count("147.04")

# use a count function to enumerate the # of deamidated N's in each peptide
comet278['n-deam'] = comet278['peptide'].str.count("115.03")

# use a count function to enumerate the # of deamidated Q's in each peptide
comet278['q-deam'] = comet278['peptide'].str.count("129.04")

# use a count function to enumerate the # of hydroxylated K's in each peptide
comet278['k-hydr'] = comet278['peptide'].str.count("144.09")

# use a count function to enumerate the # of hydroxylated P's in each peptide
# I removed P hydroxyation in final searches because there were so few
#comet278['p-hydr'] = comet278['peptide'].str.count("131.05")

# use a count function to enumerate the # of methylated R's in each peptide
comet278['r-meth'] = comet278['peptide'].str.count("170.12")

# create a column with 'stripped' peptide sequences using strip
comet278['stripped peptide'] = comet278['peptide'].str[2:].str[:-2].str.replace(r"\[.*\]","")

# add a column with the stripped peptide length (number of AAs)
comet278['stripped length'] = comet278['stripped peptide'].apply(len)

comet278['NAAF num.'] = comet278['precursor_intensity'] / comet278['stripped length']

# total the number of modifications in sequence
comet278['ptm-total'] = comet278['c-carb'] + comet278['m-oxid'] + comet278['n-deam'] + comet278['q-deam'] + comet278['k-hydr'] + comet278['r-meth']

# turn all isoleucines into leucines
# this helps later in comparing Unipept peptides to PeaksDB and Comet ones
comet278['stripped I-L'] = comet278['stripped peptide'].str.replace('I','L')

# write modified dataframe to new txt file
comet278.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_fungidb.csv")

# check out the results
comet278.head()

/home/millieginty/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,spectrum,xcorr,deltacn,expect,ions,peptide,calc_neutral_pep_mass,precursor_intensity,protein,L terminus,...,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,NAAF num.,ptm-total,stripped I-L
0,20181004_4-19_265m_topP_278_90min.45087.45087.2,5.642,0.615,7.440000e-14,27/44,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,2215090.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,96308.260870,0,ALQQQLENPLAQQLLSGELVPGK
1,20181004_4-19_265m_topP_278_90min.45016.45016.3,5.634,0.503,1.350000e-08,29/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,13100100.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,569569.565217,0,ALQQQLENPLAQQLLSGELVPGK
2,20181004_4-19_265m_topP_278_90min.24694.24694.2,4.085,0.420,3.220000e-05,22/36,R.LPQVEGTGGDVQPSQDLVR.V,1994.0069,18697900.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,984100.000000,0,LPQVEGTGGDVQPSQDLVR
3,20181004_4-19_265m_topP_278_90min.25494.25494.2,3.972,0.463,1.230000e-06,22/36,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,1994.9909,2532930.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,1,0,0,LPQVEGTGGDVQPSQDLVR,19,133312.105263,1,LPQVEGTGGDVQPSQDLVR
4,20181004_4-19_265m_topP_278_90min.45263.45263.3,3.884,1.000,8.030000e-12,19/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,376172.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,16355.304348,0,ALQQQLENPLAQQLLSGELVPGK


## Calculating the false discovery rate (% FDR)

### Filtering PSMs > a selected XCorr value and exporting peptides

In [8]:
# Let's separate out the decoy hits from the good ones

comet278pmm = comet278[~comet278['protein'].str.contains("DECOY")]
comet278dec = comet278[comet278['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet278pmm))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet278dec))

# calculate the bulk FDR (all PSMs so let's not beat ourselves up)

r = len(comet278pmm)
d = len(comet278dec)

FDR = d/r*100

print("False discovery rate = ", FDR)

TypeError: bad operand type for unary ~: 'float'

In [9]:
# keep only peptides  >3 XCorr
# need to convert Xcorr column from strings to numeric so we can use loc
# note that pmm here just means 'non decoy', it's a vestige from the accession prefixes from the MED proteins
comet278['xcorr'] = pd.to_numeric(comet278['xcorr'])

comet278_3 = comet278.loc[comet278['xcorr'] >= 2.606]

# What's the FDR?

# Let's separate out the decoy hits from the good ones

comet278pmm3 = comet278_3[~comet278_3['protein'].str.contains("DECOY")]
comet278dec3 = comet278_3[comet278_3['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet278pmm3))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet278dec3))

# calculate the FDR 

r = len(comet278pmm3)
d = len(comet278dec3)

FDR = d/(d+r)*100

print("False discovery rate = ", FDR)

# real Comet PSMs 44
# decoy Comet PSMs 0
False discovery rate =  0.0


### Exporting txt files of stripped peptides at confidence cutoffs:

In [10]:
# keep only peptide column for the 1% FDR filtered PSMs
pep278moddup = comet278pmm3[["peptide"]]

# keep only the stripped peptide column for the 1% FDR filtered PSMs
# this is what we'll use for UniPept input, etc
pep278dup = comet278pmm3[["stripped peptide"]]

# deduplicate both of these lists
pep278mod = pep278moddup.drop_duplicates()
pep278 = pep278dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 278:", len(pep278moddup))
print("Deduplicated modified peptides in 278:", len(pep278mod))
print("Total strippled peptides in 278:", len(pep278dup))
print("Deduplicated stripped peptides in 278:", len(pep278))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep278.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTM-opt_Comet15_1FDR_stripped_peptides.fas

# look at the stripped peptides
pep278.head()

Total modified peptides in 278: 44
Deduplicated modified peptides in 278: 34
Total strippled peptides in 278: 44
Deduplicated stripped peptides in 278: 31


,stripped peptide
0,AIQQQIENPLAQQILSGELVPGK
2,LPQVEGTGGDVQPSQDLVR
7,VIGQNEAVDAVSNAIR
8,SASIRIPVVPSPK
9,SYELPDGQVITIGNER


In [11]:
# also export the entire peptide file (AAs, NAAF num, everything) under 1 % FDR

comet278pmm3.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_peptides.csv")

comet278pmm3.head()

,spectrum,xcorr,deltacn,expect,ions,peptide,calc_neutral_pep_mass,precursor_intensity,protein,L terminus,...,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,NAAF num.,ptm-total,stripped I-L
0,20181004_4-19_265m_topP_278_90min.45087.45087.2,5.642,0.615,7.440000e-14,27/44,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,2215090.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,96308.260870,0,ALQQQLENPLAQQLLSGELVPGK
1,20181004_4-19_265m_topP_278_90min.45016.45016.3,5.634,0.503,1.350000e-08,29/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,13100100.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,569569.565217,0,ALQQQLENPLAQQLLSGELVPGK
2,20181004_4-19_265m_topP_278_90min.24694.24694.2,4.085,0.420,3.220000e-05,22/36,R.LPQVEGTGGDVQPSQDLVR.V,1994.0069,18697900.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,984100.000000,0,LPQVEGTGGDVQPSQDLVR
3,20181004_4-19_265m_topP_278_90min.25494.25494.2,3.972,0.463,1.230000e-06,22/36,R.LPQVEGTGGDVQ[129.04]PSQDLVR.V,1994.9909,2532930.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,1,0,0,LPQVEGTGGDVQPSQDLVR,19,133312.105263,1,LPQVEGTGGDVQPSQDLVR
4,20181004_4-19_265m_topP_278_90min.45263.45263.3,3.884,1.000,8.030000e-12,19/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,376172.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,16355.304348,0,ALQQQLENPLAQQLLSGELVPGK


## NAAF correction and exporting files with AA and PTM totals:

### Only for DECOY-filtered Comet PSMs > XCorr 3.0

In [15]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >XCorr 3)

index = ['sample total']

data = {'A': comet278pmm3['A'].sum(),
        'C': comet278pmm3['C'].sum(),
        'D': comet278pmm3['D'].sum(),
        'E': comet278pmm3['E'].sum(),
        'F': comet278pmm3['F'].sum(),
        'G': comet278pmm3['G'].sum(),
        'H': comet278pmm3['H'].sum(),
        'I': comet278pmm3['I'].sum(),
        'K': comet278pmm3['K'].sum(),
        'L': comet278pmm3['L'].sum(),
        'M': comet278pmm3['M'].sum(),
        'N': comet278pmm3['N'].sum(),
        'P': comet278pmm3['P'].sum(),
        'Q': comet278pmm3['Q'].sum(),
        'R': comet278pmm3['R'].sum(),
        'S': comet278pmm3['S'].sum(),
        'T': comet278pmm3['T'].sum(),
        'V': comet278pmm3['V'].sum(),
        'W': comet278pmm3['W'].sum(),
        'Y': comet278pmm3['Y'].sum(),
        'c-carb': comet278pmm3['c-carb'].sum(),
        'm-oxid': comet278pmm3['m-oxid'].sum(),
        'n-deam': comet278pmm3['n-deam'].sum(),
        'q-deam': comet278pmm3['q-deam'].sum(),
        'k-hydr': comet278pmm3['k-hydr'].sum(),
        'r-meth': comet278pmm3['r-meth'].sum(),
        'Total area': comet278pmm3['precursor_intensity'].sum(),
        'Total length': comet278pmm3['stripped length'].sum()
       }

totalcomet278pmm3 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalcomet278pmm3['% C w/ carb'] = totalcomet278pmm3['c-carb'] / totalcomet278pmm3['C'] 

# calculate percentage of M's that are oxidized
totalcomet278pmm3['% M w/ oxid'] = totalcomet278pmm3['m-oxid'] / totalcomet278pmm3['M'] 

# calculate percentage of N's that are deamidated
totalcomet278pmm3['% N w/ deam'] = totalcomet278pmm3['n-deam'] / totalcomet278pmm3['N'] 

# calculate percentage of Q's that are deamidated
totalcomet278pmm3['% Q w/ deam'] = totalcomet278pmm3['q-deam'] / totalcomet278pmm3['Q'] 

# calculate percentage of K's that are hydroxylated
totalcomet278pmm3['% K w/ hydr'] = totalcomet278pmm3['k-hydr'] / totalcomet278pmm3['K'] 

# calculate percentage of P's that are hydroxylated
#totalcomet278pmm3['% P w/ hydr'] = totalcomet278pmm3['p-hydr'] / totalcomet278pmm3['P'] 

# calculate percentage of R's that are methylated
totalcomet278pmm3['% R w/ meth'] = totalcomet278pmm3['r-meth'] / totalcomet278pmm3['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalcomet278pmm3['NAAF denom.'] = totalcomet278pmm3['Total area'] / totalcomet278pmm3['Total length']

# write modified dataframe to new txt file
totalcomet278pmm3.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_totals.csv")

totalcomet278pmm3.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,68,0,57,54,10,75,7,50,33,56,...,3,133323088.0,660,NaN,0.0,0.166667,0.067797,0.090909,0.055556,202004.678788


In [16]:
# use the calculated NAAF factor (in totalcomet278pmm3 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF3 = 202004.678788

# use NAAF >XCorr 3 to get NAAF
comet278pmm3['NAAF factor'] = (comet278pmm3['NAAF num.'])/NAAF3

# make a dataframe that contains only what we need: sequences, AAs, PTMs
comet278pmm3_NAAF = comet278pmm3[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', \
                                'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF3 factor by the AAs to normalize its abundance by peak area and peptide length

comet278pmm3_NAAF['A-NAAF3'] = comet278pmm3_NAAF['A'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['C-NAAF3'] = comet278pmm3_NAAF['C'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['D-NAAF3'] = comet278pmm3_NAAF['D'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['E-NAAF3'] = comet278pmm3_NAAF['E'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['F-NAAF3'] = comet278pmm3_NAAF['F'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['G-NAAF3'] = comet278pmm3_NAAF['G'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['H-NAAF3'] = comet278pmm3_NAAF['H'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['I-NAAF3'] = comet278pmm3_NAAF['I'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['K-NAAF3'] = comet278pmm3_NAAF['K'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['L-NAAF3'] = comet278pmm3_NAAF['L'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['M-NAAF3'] = comet278pmm3_NAAF['M'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['N-NAAF3'] = comet278pmm3_NAAF['N'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['P-NAAF3'] = comet278pmm3_NAAF['P'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['Q-NAAF3'] = comet278pmm3_NAAF['Q'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['R-NAAF3'] = comet278pmm3_NAAF['R'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['S-NAAF3'] = comet278pmm3_NAAF['S'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['T-NAAF3'] = comet278pmm3_NAAF['T'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['V-NAAF3'] = comet278pmm3_NAAF['V'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['W-NAAF3'] = comet278pmm3_NAAF['W'] * comet278pmm3['NAAF factor']
comet278pmm3_NAAF['Y-NAAF3'] = comet278pmm3_NAAF['Y'] * comet278pmm3['NAAF factor']

# multiply the NAAF3 factor by the PTMs normalize its abundance by peak area and peptide length

comet278pmm3_NAAF['ccarb-NAAF3'] = comet278pmm3_NAAF['c-carb'] * comet278pmm3_NAAF['NAAF factor']
comet278pmm3_NAAF['moxid-NAAF3'] = comet278pmm3_NAAF['m-oxid'] * comet278pmm3_NAAF['NAAF factor']
comet278pmm3_NAAF['ndeam-NAAF3'] = comet278pmm3_NAAF['n-deam'] * comet278pmm3_NAAF['NAAF factor']
comet278pmm3_NAAF['qdeam-NAAF3'] = comet278pmm3_NAAF['q-deam'] * comet278pmm3_NAAF['NAAF factor']
comet278pmm3_NAAF['khydr-NAAF3'] = comet278pmm3_NAAF['k-hydr'] * comet278pmm3_NAAF['NAAF factor']
comet278pmm3_NAAF['rmeth-NAAF3'] = comet278pmm3_NAAF['r-meth'] * comet278pmm3_NAAF['NAAF factor']

# write the dataframe to a new csv
comet278pmm3_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_NAAF.csv")

comet278pmm3_NAAF.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF3,V-NAAF3,W-NAAF3,Y-NAAF3,ccarb-NAAF3,moxid-NAAF3,ndeam-NAAF3,qdeam-NAAF3,khydr-NAAF3,rmeth-NAAF3
0,AIQQQIENPLAQQILSGELVPGK,0.476763,2,0,0,2,0,2,0,3,...,0.000000,0.953525,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,AIQQQIENPLAQQILSGELVPGK,2.819586,2,0,0,2,0,2,0,3,...,0.000000,5.639172,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,LPQVEGTGGDVQPSQDLVR,4.871669,0,0,2,1,0,3,0,0,...,4.871669,19.486677,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,LPQVEGTGGDVQPSQDLVR,0.659946,0,0,2,1,0,3,0,0,...,0.659946,2.639783,0.0,0.0,0.0,0.0,0.0,0.659946,0.0,0.0
4,AIQQQIENPLAQQILSGELVPGK,0.080965,2,0,0,2,0,2,0,3,...,0.000000,0.161930,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [17]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet278 Comet results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': comet278pmm3_NAAF['NAAF factor'].sum(),
        'A-NAAF': comet278pmm3_NAAF['A-NAAF3'].sum(),
        'C-NAAF': comet278pmm3_NAAF['C-NAAF3'].sum(),
        'D-NAAF': comet278pmm3_NAAF['D-NAAF3'].sum(),
        'E-NAAF': comet278pmm3_NAAF['E-NAAF3'].sum(),
        'F-NAAF': comet278pmm3_NAAF['F-NAAF3'].sum(),
        'G-NAAF': comet278pmm3_NAAF['G-NAAF3'].sum(),
        'H-NAAF': comet278pmm3_NAAF['H-NAAF3'].sum(),
        'I-NAAF': comet278pmm3_NAAF['I-NAAF3'].sum(),
        'K-NAAF': comet278pmm3_NAAF['K-NAAF3'].sum(),
        'L-NAAF': comet278pmm3_NAAF['L-NAAF3'].sum(),
        'M-NAAF': comet278pmm3_NAAF['M-NAAF3'].sum(),
        'N-NAAF': comet278pmm3_NAAF['N-NAAF3'].sum(),
        'P-NAAF': comet278pmm3_NAAF['P-NAAF3'].sum(),
        'Q-NAAF': comet278pmm3_NAAF['Q-NAAF3'].sum(),
        'R-NAAF': comet278pmm3_NAAF['R-NAAF3'].sum(),
        'S-NAAF': comet278pmm3_NAAF['S-NAAF3'].sum(),
        'T-NAAF': comet278pmm3_NAAF['T-NAAF3'].sum(),
        'V-NAAF': comet278pmm3_NAAF['V-NAAF3'].sum(),
        'W-NAAF': comet278pmm3_NAAF['W-NAAF3'].sum(),
        'Y-NAAF': comet278pmm3_NAAF['Y-NAAF3'].sum(),
        'C-carb-NAAF': comet278pmm3_NAAF['ccarb-NAAF3'].sum(),
        'M-oxid-NAAF': comet278pmm3_NAAF['moxid-NAAF3'].sum(),
        'N-deam-NAAF': comet278pmm3_NAAF['ndeam-NAAF3'].sum(),
        'Q-deam-NAAF': comet278pmm3_NAAF['qdeam-NAAF3'].sum(),
        'K-hydr-NAAF': comet278pmm3_NAAF['khydr-NAAF3'].sum(),
        'R-meth-NAAF': comet278pmm3_NAAF['rmeth-NAAF3'].sum()
       }

totalcomet278pmm3_NAAF = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


totalcomet278pmm3_NAAF['% C w/ carb. NAAF'] = totalcomet278pmm3_NAAF['C-carb-NAAF'] / totalcomet278pmm3_NAAF['C-NAAF']
totalcomet278pmm3_NAAF['% M w/ oxid. NAAF'] = totalcomet278pmm3_NAAF['M-oxid-NAAF'] / totalcomet278pmm3_NAAF['M-NAAF']
totalcomet278pmm3_NAAF['% N w/ deam. NAAF'] = totalcomet278pmm3_NAAF['N-deam-NAAF'] / totalcomet278pmm3_NAAF['N-NAAF']
totalcomet278pmm3_NAAF['% Q w/ deam. NAAF'] = totalcomet278pmm3_NAAF['Q-deam-NAAF'] / totalcomet278pmm3_NAAF['Q-NAAF']
totalcomet278pmm3_NAAF['% K w/ hydr. NAAF'] = totalcomet278pmm3_NAAF['K-hydr-NAAF'] / totalcomet278pmm3_NAAF['K-NAAF']
totalcomet278pmm3_NAAF['% R w/ meth. NAAF'] = totalcomet278pmm3_NAAF['R-meth-NAAF'] / totalcomet278pmm3_NAAF['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalcomet278pmm3_NAAF['NAAF check'] = totalcomet278pmm3_NAAF['NAAF'] / 202004.678788

# write modified dataframe to new txt file, same name + totals
totalcomet278pmm3_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/278/ETNP-SKQ17-278-265m-trap_PTMopt_Comet15_1FDR_NAAF_totals.csv")

totalcomet278pmm3_NAAF.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,57.568698,85.059409,0.0,51.63332,92.967246,7.673069,84.396103,16.673407,46.36742,46.01633,...,1.668686,14.74329,16.766056,NaN,0.0,0.444132,0.028681,0.320393,0.236577,0.000285


### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()